# Multiple runs

## Perform Multiple Optimization Runs with EnergyScope

In this tutorial, we will demonstrate how to perform multiple optimization runs using the EnergyScope model. This is useful for sensitivity analysis, scenario exploration, and understanding how changes in parameters affect the energy system configuration.

### Import Necessary Libraries

We begin by importing the required libraries and modules:

In [1]:
import pandas as pd
import pickle 
from energyscope.energyscope import Energyscope
from energyscope.models import infrastructure_ch_2050
from energyscope.result import postprocessing
from energyscope.plots import plot_sankey, plot_parametrisation



- **`pandas`**: For data manipulation and handling data frames.
- **`pickle`**: For saving and loading Python objects to and from files.
- **`Energyscope`**: The main class for initializing and running the EnergyScope model.
- **`infrastructure_ch_2050`**: A predefined model configuration focusing on energy infrastructure In Switzerland in 2050.
- **`postprocessing`**: Functions for processing and analyzing results after optimization.
- **`plot_sankey`**, **`plot_parametrisation`**: Functions for visualizing results.

### Define Solver Options

We specify the solver options to control the optimization process:

In [2]:
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}



- **`'solver': 'gurobi'`**: Specifies that the Gurobi solver should be used.
- **`'solver_msg': 0`**: Suppresses solver messages during execution.

### Initialize and Run the Base Model

We initialize the EnergyScope model with the chosen dataset and solver options:

In [3]:
# Load the model with the chosen dataset and solver options
es_infra_ch = Energyscope(model=infrastructure_ch_2050, solver_options=solver_options)



Then, we perform an initial calculation to ensure the model is set up correctly:

In [4]:
# Solve the model
results_ch = es_infra_ch.calc()

Gurobi 11.0.0:



> **Note**: This initial run is optional but recommended to verify that the model and solver are functioning properly before proceeding to multiple runs.

### Load Parameter Sequence Data

We load a sequence of parameters from an Excel file, which will be used to perform multiple optimization runs:

In [5]:
# Load the parameter sequence DataFrame
seq_data = pd.read_excel("tutorial_input/param_run_es_n_infrastructure_ch_2050.xlsx")
display(seq_data)

,param,index0,index1,index2,index3,value1,value2,value3,value4,value5,value6,value7,value8,value9,value10,value11
0,f_min,PV,NaN,NaN,NaN,2,2.60,5.20,7.80,10.40,13.00,15.60,18.20,20.80,23.40,26.00
1,f_max,PV,NaN,NaN,NaN,2,2.60,5.20,7.80,10.40,13.00,15.60,18.20,20.80,23.40,26.00
2,end_uses_demand_year,MOBILITY_FREIGHT,TRANSPORTATION,NaN,NaN,45000,33226.71,33226.71,33226.71,33226.71,33226.71,33226.71,33226.71,33226.71,33226.71,33226.71
3,c_inv,WIND,NaN,NaN,NaN,800,850.00,900.00,950.00,1000.00,1050.00,1100.00,1150.00,1200.00,1250.00,1300.00




- **`seq_data`**: A DataFrame containing different sets of parameters for each run.
- **`display(seq_data)`**: Displays the DataFrame to inspect the parameters being varied.

### Perform Multiple Optimization Runs

We use the `calc_sequence` method to run the model multiple times based on the parameter changes specified in `seq_data`:

In [6]:
# Run multiple optimizations based on parameters changed in seq_data
results_ch_n = es_infra_ch.calc_sequence(seq_data)

Gurobi 11.0.0:Gurobi 11.0.0:1
Gurobi 11.0.0:2
Gurobi 11.0.0:3
Gurobi 11.0.0:4
Gurobi 11.0.0:5
Gurobi 11.0.0:6
Gurobi 11.0.0:7
Gurobi 11.0.0:8
Gurobi 11.0.0:9
Gurobi 11.0.0:10
Gurobi 11.0.0:11




- **`results_ch_n`**: A `Result` object that contains the outputs of all runs.

### Post-Process the Results

After obtaining the results from multiple runs, we apply post-processing to compute Key Performance Indicators (KPIs) and prepare the data for visualization:

In [7]:
# Postcompute KPIs
results_ch_n = postprocessing(results_ch_n)



### Visualize Results with Sankey Diagrams

We can visualize the energy flows for specific runs using Sankey diagrams.

#### Generate and Display Sankey Diagram for Run 1

In [8]:
# Generate the Sankey diagram for run 1
fig = plot_sankey(results_ch_n, run_id=1)
fig.show()



- **`run_id=1`**: Specifies that we want to visualize the results from the first run.

#### Generate and Display Sankey Diagram for Run 11

In [9]:
# Generate the Sankey diagram for run 11
fig = plot_sankey(results_ch_n, run_id=11)
fig.show()



- **`run_id=11`**: Visualizes the results from the eleventh run.

> **Optional**: You can save the generated Sankey diagrams as HTML files or images by uncommenting and modifying the following lines:

In [10]:
# Save the generated Sankey diagram as an HTML file
# fig.write_html("tutorial_output/Sankey_results_ch_1.html")

# Save the generated Sankey diagram as an image
# fig.write_image('tutorial_output/Sankey_results_ch_1.png')



### Analyze Parametrization Results

We can further analyze how the changes in parameters affect the results using the `plot_parametrisation` function.

#### Inspect Annual Results DataFrame

In [11]:
# Display a sample from the annual results DataFrame
display(results_ch_n.postprocessing['df_annual'].sample())

,,C_inv,C_maint,Annual_Prod,F_Mult,tau,C_inv_an,Category,Category_2,Sector
,Run,,,,,,,,,
CAR_ETOH_E10,9,0.0,0.0,0.0,0.0,0.137778,0.0,Others,Mobility,Others




- Displays a random sample from the annual results DataFrame for inspection.

#### Plot Investment Costs by Sector

In [12]:
# Plot annualized investment costs aggregated by sector
plot_parametrisation(results=results_ch_n, variable="C_inv_an", category="Sector", 
                     labels = {"Run": "Simulation Run","C_inv_an": "Annualized investment costs [MCHF/y]"})



- **`variable="C_inv_an"`**: Specifies that we want to plot annual investment costs.
- **`category="Sector"`**: Aggregates the costs by sector.

#### Plot Investment Costs by Category


In [13]:
# Plot annualized investment costs aggregated by category
plot_parametrisation(results=results_ch_n, variable="C_inv_an", category="Category")



- Aggregates the costs by category.

#### Plot Investment Costs by Sub-Category

In [14]:
# Plot annualized investment costs aggregated by sub-category
plot_parametrisation(results=results_ch_n, variable="C_inv_an", category="Category_2")



- **`Category_2`**: A more detailed level of categorization, allowing for a finer analysis.

### Save and Load Results

To preserve the results of the multiple runs and avoid re-running the optimizations, we can save the results to a file and load them later.

#### Define Save Function

In [15]:
def save_result_to_pickle(data, filename):
    """
    Save the Result object to a pickle file.

    Parameters:
        data: The Result object to save.
        filename (str): The file path to save the object to.
    """
    with open(filename, 'wb') as fp:
        pickle.dump(data, fp, protocol=pickle.HIGHEST_PROTOCOL)



#### Define Load Function

In [16]:
def load_result_from_pickle(filename):
    """
    Load the Result object from a pickle file.

    Parameters:
        filename (str): The file path to load the object from.

    Returns:
        The loaded Result object.
    """
    with open(filename, 'rb') as handle:
        result = pickle.load(handle)
    return result



> **Note**: These utility functions could be integrated into the EnergyScope library for convenience.

#### Save the Results

In [17]:
# Save the result object to a pickle file
save_result_to_pickle(results_ch_n, "tutorial_input/results_ch_n.pickle")



#### Clear the Results Variable

In [18]:
# Empty the variable to simulate a fresh environment
results_ch_n = None 



#### Load the Results

In [19]:
# Load the saved result from the pickle file
results_ch_n = load_result_from_pickle("tutorial_input/results_ch_n.pickle")



#### Display Total Cost

In [20]:
# Show the total cost from the loaded results
results_ch_n.variables['TotalCost']

,TotalCost,Run
0,7462.638856,1
0,7481.066542,2
0,7602.302609,3
0,7727.531811,4
0,7852.761013,5
0,7978.875140,6
0,8106.176886,7
0,8233.669615,8
0,8355.658041,9
0,8467.853665,10




- Accesses and displays the total cost from each run in the loaded results, verifying that the data was correctly saved and loaded.

---

By following these steps, you can:

- **Perform multiple optimization runs** with varying parameters to analyze different scenarios.
- **Visualize the results** of specific runs using Sankey diagrams, providing insight into energy flows.
- **Analyze the impact** of parameter changes on key variables like investment costs through parametrization plots.
- **Save and load the results** for future analysis, enhancing reproducibility and efficiency.

This approach is particularly useful for conducting sensitivity analyses, exploring different energy strategies, and gaining deeper insights into the energy system's behavior under various conditions.

> **Note**: Ensure that the Excel file `"tutorial_input/param_run_es_n_infrastructure_ch_2050.xlsx"` and the pickle file paths are correctly set in your environment. Additionally, the `plot_parametrisation` function may require specific data structures; refer to the EnergyScope documentation for more details.

By leveraging these techniques, you can effectively utilize the EnergyScope model for comprehensive energy system analysis.